In [ ]:
import pickle
import ast
import re
import nltk                                                                                                                     # type: ignore
from nltk.corpus import stopwords                                                                                               # type: ignore
import numpy as np                                                                                                              # type: ignore
import pandas as pd                                                                                                             # type: ignore
import matplotlib.pyplot as plt                                                                                                 # type: ignore
import seaborn as sns                                                                                                   z        # type: ignore
from nltk.stem.porter import PorterStemmer                                                                                      # type: ignore
from sklearn.feature_extraction.text import CountVectorizer                                                                     # type: ignore
from sklearn.metrics.pairwise import cosine_similarity                                                                          # type: ignore
import warnings
warnings.filterwarnings('ignore')

In [13]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits= pd.read_csv('tmdb_5000_credits.csv')

movies.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


In [3]:
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [4]:
# Merging the dataframes

movies = movies.merge(credits,on='title')

In [5]:
movies.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


In [6]:
df = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]
df.head(3)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


## Now the data should be converted to normal form

In [7]:
lst = ast.literal_eval(df['genres'][0])
lst

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [8]:
def get_genres(text):
    l = []
    for i in ast.literal_eval(text):
        l.append(i['name'])
    return l

In [9]:
df['genres'] = df['genres'].apply(get_genres)

In [10]:
lst = ast.literal_eval(df['keywords'][0])
lst

[{'id': 1463, 'name': 'culture clash'},
 {'id': 2964, 'name': 'future'},
 {'id': 3386, 'name': 'space war'},
 {'id': 3388, 'name': 'space colony'},
 {'id': 3679, 'name': 'society'},
 {'id': 3801, 'name': 'space travel'},
 {'id': 9685, 'name': 'futuristic'},
 {'id': 9840, 'name': 'romance'},
 {'id': 9882, 'name': 'space'},
 {'id': 9951, 'name': 'alien'},
 {'id': 10148, 'name': 'tribe'},
 {'id': 10158, 'name': 'alien planet'},
 {'id': 10987, 'name': 'cgi'},
 {'id': 11399, 'name': 'marine'},
 {'id': 13065, 'name': 'soldier'},
 {'id': 14643, 'name': 'battle'},
 {'id': 14720, 'name': 'love affair'},
 {'id': 165431, 'name': 'anti war'},
 {'id': 193554, 'name': 'power relations'},
 {'id': 206690, 'name': 'mind and soul'},
 {'id': 209714, 'name': '3d'}]

In [11]:
def get_keywords(name):
    l=[]
    for i in ast.literal_eval(name):
        l.append(i['name'])

    return l

In [12]:
df['keywords'] = df['keywords'].apply(get_keywords)
df['keywords']

KeyboardInterrupt: 

In [595]:
lst = ast.literal_eval(df['cast'][0])
lst

[{'cast_id': 242,
  'character': 'Jake Sully',
  'credit_id': '5602a8a7c3a3685532001c9a',
  'gender': 2,
  'id': 65731,
  'name': 'Sam Worthington',
  'order': 0},
 {'cast_id': 3,
  'character': 'Neytiri',
  'credit_id': '52fe48009251416c750ac9cb',
  'gender': 1,
  'id': 8691,
  'name': 'Zoe Saldana',
  'order': 1},
 {'cast_id': 25,
  'character': 'Dr. Grace Augustine',
  'credit_id': '52fe48009251416c750aca39',
  'gender': 1,
  'id': 10205,
  'name': 'Sigourney Weaver',
  'order': 2},
 {'cast_id': 4,
  'character': 'Col. Quaritch',
  'credit_id': '52fe48009251416c750ac9cf',
  'gender': 2,
  'id': 32747,
  'name': 'Stephen Lang',
  'order': 3},
 {'cast_id': 5,
  'character': 'Trudy Chacon',
  'credit_id': '52fe48009251416c750ac9d3',
  'gender': 1,
  'id': 17647,
  'name': 'Michelle Rodriguez',
  'order': 4},
 {'cast_id': 8,
  'character': 'Selfridge',
  'credit_id': '52fe48009251416c750ac9e1',
  'gender': 2,
  'id': 1771,
  'name': 'Giovanni Ribisi',
  'order': 5},
 {'cast_id': 7,
  'c

In [596]:
def get_cast(name):
    l=[]
    cnt=0
    for i in ast.literal_eval(name):
        if cnt >=5:
            break
        l.append(i['name'])
        cnt+=1

    return l

In [597]:
df['cast'] = df['cast'].apply(get_cast)
df['cast'][0]

['Sam Worthington',
 'Zoe Saldana',
 'Sigourney Weaver',
 'Stephen Lang',
 'Michelle Rodriguez']

In [598]:
lst = ast.literal_eval(df['crew'][0])
lst

[{'credit_id': '52fe48009251416c750aca23',
  'department': 'Editing',
  'gender': 0,
  'id': 1721,
  'job': 'Editor',
  'name': 'Stephen E. Rivkin'},
 {'credit_id': '539c47ecc3a36810e3001f87',
  'department': 'Art',
  'gender': 2,
  'id': 496,
  'job': 'Production Design',
  'name': 'Rick Carter'},
 {'credit_id': '54491c89c3a3680fb4001cf7',
  'department': 'Sound',
  'gender': 0,
  'id': 900,
  'job': 'Sound Designer',
  'name': 'Christopher Boyes'},
 {'credit_id': '54491cb70e0a267480001bd0',
  'department': 'Sound',
  'gender': 0,
  'id': 900,
  'job': 'Supervising Sound Editor',
  'name': 'Christopher Boyes'},
 {'credit_id': '539c4a4cc3a36810c9002101',
  'department': 'Production',
  'gender': 1,
  'id': 1262,
  'job': 'Casting',
  'name': 'Mali Finn'},
 {'credit_id': '5544ee3b925141499f0008fc',
  'department': 'Sound',
  'gender': 2,
  'id': 1729,
  'job': 'Original Music Composer',
  'name': 'James Horner'},
 {'credit_id': '52fe48009251416c750ac9c3',
  'department': 'Directing',
  

In [599]:
def get_crew(name):
    l = []
    for i in ast.literal_eval(name):
        if i['job']=='Director':
            l.append(i['name'])
        
    return l

In [600]:
df['crew'] = df['crew'].apply(get_crew)

In [601]:
df['crew'][0]

['James Cameron']

In [602]:
df

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weave...",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley, ...",[Gore Verbinski]
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux, R...",[Sam Mendes]
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman, A...",[Christopher Nolan]
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton,...",[Andrew Stanton]
...,...,...,...,...,...,...,...
4804,9367,El Mariachi,El Mariachi just wants to play his guitar and ...,"[Action, Crime, Thriller]","[united states–mexico barrier, legs, arms, pap...","[Carlos Gallardo, Jaime de Hoyos, Peter Marqua...",[Robert Rodriguez]
4805,72766,Newlyweds,A newlywed couple's honeymoon is upended by th...,"[Comedy, Romance]",[],"[Edward Burns, Kerry Bishé, Marsha Dietlein, C...",[Edward Burns]
4806,231617,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...","[Comedy, Drama, Romance, TV Movie]","[date, love at first sight, narration, investi...","[Eric Mabius, Kristin Booth, Crystal Lowe, Geo...",[Scott Smith]
4807,126186,Shanghai Calling,When ambitious New York attorney Sam is sent t...,[],[],"[Daniel Henney, Eliza Coupe, Bill Paxton, Alan...",[Daniel Hsia]


In [603]:
# df['overview'].map(lambda x:x.split())
# it is giving an error so there might be some missing values in the overview

df.isna().sum()
df.dropna(inplace=True)

In [604]:
df['overview'] = df['overview'].map(lambda x:x.split())

In [605]:
df

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weave...",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley, ...",[Gore Verbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux, R...",[Sam Mendes]
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman, A...",[Christopher Nolan]
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton,...",[Andrew Stanton]
...,...,...,...,...,...,...,...
4804,9367,El Mariachi,"[El, Mariachi, just, wants, to, play, his, gui...","[Action, Crime, Thriller]","[united states–mexico barrier, legs, arms, pap...","[Carlos Gallardo, Jaime de Hoyos, Peter Marqua...",[Robert Rodriguez]
4805,72766,Newlyweds,"[A, newlywed, couple's, honeymoon, is, upended...","[Comedy, Romance]",[],"[Edward Burns, Kerry Bishé, Marsha Dietlein, C...",[Edward Burns]
4806,231617,"Signed, Sealed, Delivered","[""Signed,, Sealed,, Delivered"", introduces, a,...","[Comedy, Drama, Romance, TV Movie]","[date, love at first sight, narration, investi...","[Eric Mabius, Kristin Booth, Crystal Lowe, Geo...",[Scott Smith]
4807,126186,Shanghai Calling,"[When, ambitious, New, York, attorney, Sam, is...",[],[],"[Daniel Henney, Eliza Coupe, Bill Paxton, Alan...",[Daniel Hsia]


In [606]:
df = pd.concat([df,df['overview'] + df['genres'] + df['keywords'] + df['cast'] + df['crew']],axis=1)

In [607]:
df.rename(columns={0:'Tags'},inplace=True)

In [608]:
df['Tags'][0]

['In',
 'the',
 '22nd',
 'century,',
 'a',
 'paraplegic',
 'Marine',
 'is',
 'dispatched',
 'to',
 'the',
 'moon',
 'Pandora',
 'on',
 'a',
 'unique',
 'mission,',
 'but',
 'becomes',
 'torn',
 'between',
 'following',
 'orders',
 'and',
 'protecting',
 'an',
 'alien',
 'civilization.',
 'Action',
 'Adventure',
 'Fantasy',
 'Science Fiction',
 'culture clash',
 'future',
 'space war',
 'space colony',
 'society',
 'space travel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alien planet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'love affair',
 'anti war',
 'power relations',
 'mind and soul',
 '3d',
 'Sam Worthington',
 'Zoe Saldana',
 'Sigourney Weaver',
 'Stephen Lang',
 'Michelle Rodriguez',
 'James Cameron']

In [609]:
df.drop(columns=['overview','genres','keywords','cast','crew'],inplace=True)

In [610]:
df

,movie_id,title,Tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."
...,...,...,...
4804,9367,El Mariachi,"[El, Mariachi, just, wants, to, play, his, gui..."
4805,72766,Newlyweds,"[A, newlywed, couple's, honeymoon, is, upended..."
4806,231617,"Signed, Sealed, Delivered","[""Signed,, Sealed,, Delivered"", introduces, a,..."
4807,126186,Shanghai Calling,"[When, ambitious, New, York, attorney, Sam, is..."


In [611]:
print(df['Tags'][0])

['In', 'the', '22nd', 'century,', 'a', 'paraplegic', 'Marine', 'is', 'dispatched', 'to', 'the', 'moon', 'Pandora', 'on', 'a', 'unique', 'mission,', 'but', 'becomes', 'torn', 'between', 'following', 'orders', 'and', 'protecting', 'an', 'alien', 'civilization.', 'Action', 'Adventure', 'Fantasy', 'Science Fiction', 'culture clash', 'future', 'space war', 'space colony', 'society', 'space travel', 'futuristic', 'romance', 'space', 'alien', 'tribe', 'alien planet', 'cgi', 'marine', 'soldier', 'battle', 'love affair', 'anti war', 'power relations', 'mind and soul', '3d', 'Sam Worthington', 'Zoe Saldana', 'Sigourney Weaver', 'Stephen Lang', 'Michelle Rodriguez', 'James Cameron']


### Now we have to replace spaces so that computer can easily understand it
#### for example computer will treat james cameron as two different words where as both are part of same thing.
#### so we will replace all the space with empty

In [612]:
df['Tags'] = df['Tags'].apply(lambda x: [i.replace(" ", '') for i in x])
df

,movie_id,title,Tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."
...,...,...,...
4804,9367,El Mariachi,"[El, Mariachi, just, wants, to, play, his, gui..."
4805,72766,Newlyweds,"[A, newlywed, couple's, honeymoon, is, upended..."
4806,231617,"Signed, Sealed, Delivered","[""Signed,, Sealed,, Delivered"", introduces, a,..."
4807,126186,Shanghai Calling,"[When, ambitious, New, York, attorney, Sam, is..."


In [613]:
print(df['Tags'][0])

['In', 'the', '22nd', 'century,', 'a', 'paraplegic', 'Marine', 'is', 'dispatched', 'to', 'the', 'moon', 'Pandora', 'on', 'a', 'unique', 'mission,', 'but', 'becomes', 'torn', 'between', 'following', 'orders', 'and', 'protecting', 'an', 'alien', 'civilization.', 'Action', 'Adventure', 'Fantasy', 'ScienceFiction', 'cultureclash', 'future', 'spacewar', 'spacecolony', 'society', 'spacetravel', 'futuristic', 'romance', 'space', 'alien', 'tribe', 'alienplanet', 'cgi', 'marine', 'soldier', 'battle', 'loveaffair', 'antiwar', 'powerrelations', 'mindandsoul', '3d', 'SamWorthington', 'ZoeSaldana', 'SigourneyWeaver', 'StephenLang', 'MichelleRodriguez', 'JamesCameron']


#### now that the unnecessary spaces are removed, we have to join all of it to make a single string so that we can convert it to numeric form.

In [614]:
# # Convert each list of strings into a single string and then wrap it in a list
# df['Tags'] = df['Tags'].apply(lambda x: [' '.join(x)])

# # Check the result
# print(df['Tags'].head())


In [615]:
df['Tags']

0       [In, the, 22nd, century,, a, paraplegic, Marin...
1       [Captain, Barbossa,, long, believed, to, be, d...
2       [A, cryptic, message, from, Bond’s, past, send...
3       [Following, the, death, of, District, Attorney...
4       [John, Carter, is, a, war-weary,, former, mili...
                              ...                        
4804    [El, Mariachi, just, wants, to, play, his, gui...
4805    [A, newlywed, couple's, honeymoon, is, upended...
4806    ["Signed,, Sealed,, Delivered", introduces, a,...
4807    [When, ambitious, New, York, attorney, Sam, is...
4808    [Ever, since, the, second, grade, when, he, fi...
Name: Tags, Length: 4806, dtype: object

## Using NLP to process data
- remove capital letters
- tokenization
- stemming
- remove stopwords

## Stemming
### Stemming chops the last part of the words so that the simmilar words can be brought together
#### for example: act => action, actionable, acting

### Now we will do rest of text preprocessing with a single function

In [616]:
df['Tags'][0]

['In',
 'the',
 '22nd',
 'century,',
 'a',
 'paraplegic',
 'Marine',
 'is',
 'dispatched',
 'to',
 'the',
 'moon',
 'Pandora',
 'on',
 'a',
 'unique',
 'mission,',
 'but',
 'becomes',
 'torn',
 'between',
 'following',
 'orders',
 'and',
 'protecting',
 'an',
 'alien',
 'civilization.',
 'Action',
 'Adventure',
 'Fantasy',
 'ScienceFiction',
 'cultureclash',
 'future',
 'spacewar',
 'spacecolony',
 'society',
 'spacetravel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alienplanet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'loveaffair',
 'antiwar',
 'powerrelations',
 'mindandsoul',
 '3d',
 'SamWorthington',
 'ZoeSaldana',
 'SigourneyWeaver',
 'StephenLang',
 'MichelleRodriguez',
 'JamesCameron']

In [617]:
df['Tags'] = df['Tags'].apply(lambda x:" ".join(x))

In [618]:
df['Tags'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver StephenLang MichelleRodriguez JamesCameron'

In [619]:
ps = PorterStemmer()
def data_preprocessing(text):
    new_data = []
    for i in text.split():
        new_data.append(ps.stem(i))

    return new_data

In [620]:
df['Tags'] = df['Tags'].apply(data_preprocessing)

In [621]:
df['Tags'][0]

['in',
 'the',
 '22nd',
 'century,',
 'a',
 'parapleg',
 'marin',
 'is',
 'dispatch',
 'to',
 'the',
 'moon',
 'pandora',
 'on',
 'a',
 'uniqu',
 'mission,',
 'but',
 'becom',
 'torn',
 'between',
 'follow',
 'order',
 'and',
 'protect',
 'an',
 'alien',
 'civilization.',
 'action',
 'adventur',
 'fantasi',
 'sciencefict',
 'cultureclash',
 'futur',
 'spacewar',
 'spacecoloni',
 'societi',
 'spacetravel',
 'futurist',
 'romanc',
 'space',
 'alien',
 'tribe',
 'alienplanet',
 'cgi',
 'marin',
 'soldier',
 'battl',
 'loveaffair',
 'antiwar',
 'powerrel',
 'mindandsoul',
 '3d',
 'samworthington',
 'zoesaldana',
 'sigourneyweav',
 'stephenlang',
 'michellerodriguez',
 'jamescameron']

In [622]:
df['Tags'] = df['Tags'].apply(lambda x:" ".join(x))

In [623]:
df['Tags'][0]

'in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization. action adventur fantasi sciencefict cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d samworthington zoesaldana sigourneyweav stephenlang michellerodriguez jamescameron'

### Do not touch the above code of text processing i have spent a whole day in it

In [624]:
df

,movie_id,title,Tags
0,19995,Avatar,"in the 22nd century, a parapleg marin is dispa..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believ to be dead, ha c..."
2,206647,Spectre,a cryptic messag from bond’ past send him on a...
3,49026,The Dark Knight Rises,follow the death of district attorney harvey d...
4,49529,John Carter,"john carter is a war-weary, former militari ca..."
...,...,...,...
4804,9367,El Mariachi,el mariachi just want to play hi guitar and ca...
4805,72766,Newlyweds,a newlyw couple' honeymoon is upend by the arr...
4806,231617,"Signed, Sealed, Delivered","""signed, sealed, delivered"" introduc a dedic q..."
4807,126186,Shanghai Calling,when ambiti new york attorney sam is sent to s...


## Bag Of Words 
#### for converting the text data into numerical data

In [625]:
cv = CountVectorizer(max_features=5000, stop_words='english')
cv

CountVectorizer(max_features=5000, stop_words='english')

In [628]:
vec = cv.fit_transform(df['Tags']).toarray()

In [630]:
vec

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [639]:
for i in cv.get_feature_names_out(vec):
    print(i)

000
007
10
100
11
12
13
14
15
16
17
18
18th
19
1910
1920
1930
1940
1950
1950s
1960
1960s
1970
1970s
1980
1985
1990
19th
19thcenturi
20
200
2009
20th
24
25
30
300
3d
40
50
500
60
70
aaron
aaroneckhart
aarontaylor
abandon
abduct
abigailbreslin
abil
abl
aboard
abov
abus
academ
academi
accept
access
accid
accident
acclaim
accompani
accomplish
account
accus
ace
achiev
acquaint
act
action
actionhero
activ
activist
activities
actor
actress
actual
ad
adam
adambrodi
adamsandl
adamscott
adamshankman
adapt
add
addict
adjust
admir
admit
adolesc
adopt
ador
adrienbrodi
adult
adultanim
adulteri
adulthood
advanc
adventur
adventure
adventures
advertis
advic
advis
affair
affect
afghanistan
africa
african
africanamerican
aftercreditssting
afterlif
aftermath
ag
age
agediffer
agency
agenda
agent
agents
aggress
ago
agre
ahead
aid
aidanquinn
ail
aim
air
airplan
airplanecrash
airport
aka
al
alabama
alan
alanalda
alanarkin
alanrickman
alaska
albert
albertbrook
alcohol
alecbaldwin
alex
alexisbledel
alexkendrick

In [640]:
similarity = cosine_similarity(vec)

In [646]:
similarity

array([[1.        , 0.08111071, 0.08471737, ..., 0.04543109, 0.        ,
        0.        ],
       [0.08111071, 1.        , 0.05802589, ..., 0.023338  , 0.        ,
        0.02541643],
       [0.08471737, 0.05802589, 1.        , ..., 0.02437575, 0.        ,
        0.        ],
       ...,
       [0.04543109, 0.023338  , 0.02437575, ..., 1.        , 0.04222003,
        0.04270814],
       [0.        , 0.        , 0.        , ..., 0.04222003, 1.        ,
        0.0919601 ],
       [0.        , 0.02541643, 0.        , ..., 0.04270814, 0.0919601 ,
        1.        ]])

In [649]:
sorted(similarity[0],reverse=True)

[np.float64(1.0000000000000002),
 np.float64(0.28878211074619464),
 np.float64(0.257841025556124),
 np.float64(0.25391668753850405),
 np.float64(0.24511108480187255),
 np.float64(0.2433321316961438),
 np.float64(0.24182541670333726),
 np.float64(0.2366242621401505),
 np.float64(0.23179316248638276),
 np.float64(0.2294157338705618),
 np.float64(0.22710998958306755),
 np.float64(0.22264681319788207),
 np.float64(0.22184215999396073),
 np.float64(0.21459876881973805),
 np.float64(0.21239769762143662),
 np.float64(0.20942695414584772),
 np.float64(0.20647416048350561),
 np.float64(0.20602141085758227),
 np.float64(0.20519567041703082),
 np.float64(0.20437977982832192),
 np.float64(0.20335531332779735),
 np.float64(0.2029530274475215),
 np.float64(0.2024645717996314),
 np.float64(0.20073876713674155),
 np.float64(0.20073876713674155),
 np.float64(0.19867985355975665),
 np.float64(0.19767387315371682),
 np.float64(0.1976738731537168),
 np.float64(0.1956463952178074),
 np.float64(0.1946657053

#### But how do we know the most similar movie's name
#### SO for that we have to give an index to it

In [655]:
list(enumerate(similarity[0]))

[(0, np.float64(1.0000000000000002)),
 (1, np.float64(0.08111071056538127)),
 (2, np.float64(0.08471737420873576)),
 (3, np.float64(0.07254762501100116)),
 (4, np.float64(0.1853959098637286)),
 (5, np.float64(0.10838874619051501)),
 (6, np.float64(0.04024218182927669)),
 (7, np.float64(0.14350946197048198)),
 (8, np.float64(0.0582716546748065)),
 (9, np.float64(0.09567297464698798)),
 (10, np.float64(0.10012523486435178)),
 (11, np.float64(0.09365858115816941)),
 (12, np.float64(0.08885233166386385)),
 (13, np.float64(0.04456565178215865)),
 (14, np.float64(0.126673647984535)),
 (15, np.float64(0.06882472016116853)),
 (16, np.float64(0.07894736842105264)),
 (17, np.float64(0.1377056581888109)),
 (18, np.float64(0.09493290614465533)),
 (19, np.float64(0.0837707816583391)),
 (20, np.float64(0.05757929360284594)),
 (21, np.float64(0.10526315789473685)),
 (22, np.float64(0.06765100914917384)),
 (23, np.float64(0.08603090020146065)),
 (24, np.float64(0.0533380747062665)),
 (25, np.float64(0

In [661]:
# List of all movies similarity with avatar in decending order along with their index
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])

[(0, np.float64(1.0000000000000002)),
 (1214, np.float64(0.28878211074619464)),
 (507, np.float64(0.257841025556124)),
 (3728, np.float64(0.25391668753850405)),
 (582, np.float64(0.24511108480187255)),
 (539, np.float64(0.2433321316961438)),
 (2405, np.float64(0.24182541670333726)),
 (1202, np.float64(0.2366242621401505)),
 (61, np.float64(0.23179316248638276)),
 (778, np.float64(0.2294157338705618)),
 (1192, np.float64(0.22710998958306755)),
 (1916, np.float64(0.22264681319788207)),
 (4046, np.float64(0.22184215999396073)),
 (2782, np.float64(0.21459876881973805)),
 (172, np.float64(0.21239769762143662)),
 (1440, np.float64(0.20942695414584772)),
 (2329, np.float64(0.20647416048350561)),
 (2969, np.float64(0.20602141085758227)),
 (322, np.float64(0.20519567041703082)),
 (3606, np.float64(0.20437977982832192)),
 (1087, np.float64(0.20335531332779735)),
 (4190, np.float64(0.2029530274475215)),
 (74, np.float64(0.2024645717996314)),
 (260, np.float64(0.20073876713674155)),
 (972, np.floa

In [670]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x: x[1])[1:6]

[(1214, np.float64(0.28878211074619464)),
 (507, np.float64(0.257841025556124)),
 (3728, np.float64(0.25391668753850405)),
 (582, np.float64(0.24511108480187255)),
 (539, np.float64(0.2433321316961438))]

# The Final Function

In [ ]:
#df[df['title'] == 'Iron Man'].index[0]

In [744]:
def recommend(movie):
    #fetching index
    mv_index = df[df['title'] == movie].index[0]

    #gets a list of top 5 movies
    top5_list = (sorted(enumerate(list(similarity[mv_index])),reverse=True,key=lambda x: x[1])[1:6])

    for i in top5_list:
        print(df.iloc[i[0]].title)
    # #fetching just the index of similar movies and ignoring similarity values
    # top5 = []
    # for i in top5_list:
    #     top5.append(i[0])

    # #returning the title of  those 5 movies
    # return df.iloc[top5].title

In [745]:
recommend("Avatar")

Aliens vs Predator: Requiem
Independence Day
Falcon Rising
Battle: Los Angeles
Titan A.E.


In [748]:
recommend("Pirates of the Caribbean: At World's End")

Pirates of the Caribbean: Dead Man's Chest
Pirates of the Caribbean: The Curse of the Black Pearl
Pirates of the Caribbean: On Stranger Tides
20,000 Leagues Under the Sea
Life of Pi


In [749]:
pickle.dump(df.to_dict(), open('data_to_dict.plk', 'wb'))

In [750]:
pickle.dump(similarity, open('similarity.plk', 'wb'))